In [1]:
#Necesary imports.
import bitso
import pandas as pd
import numpy as np
from datetime import timezone
import datetime
from dateutil.tz import tzutc
import time
from fbprophet import Prophet
import matplotlib.pyplot as plt
from scipy.stats import boxcox
import warnings
warnings.filterwarnings('ignore')
import plotly.offline
from plotly import graph_objs as go

In [2]:
#Opening my api key and secret.
with open('../key.txt') as f:
    a = f.readlines()

with open('../secret.txt') as p:
    b = p.readlines()

#Using the api secret and key to call the api.
API_SECRET = [i.split('\n') for i in a][0][0]
API_KEY = [i.split('\n') for i in b][0][0]
api = bitso.Api(API_KEY, API_SECRET)

In [3]:
'''
marker for last_scrap on 31-1-2020: 13396280, I save this marker so i can use it
the next time i'm scrapping
'''
#Defining the empyt lists of the elements we need from the api.
btc_mxn_date = []
btc_mxn_price = []
btc_mxn_volume = []
btc_mxn_hour = []
btc_mxn_side = []

#Marker than changes the time in the api calls.
mark = 13396280

#Datetime when the code is running
now = datetime.datetime.now(timezone.utc)

#While loop for scrapping the information we need
while 1:
    #limit of requests, the max we can get is 100
    limit = '100'
    #Calling the api trades, we use sort 'asc' so i't starts on the marker we want.
    trades = api.trades('btc_mxn', sort='asc', limit=limit, marker=mark)
    #Starting the counter so we don't raise the error of too many requests on the api.
    count = 0
    #printing the mark and the spaces, we need these so we know the last mark we use.
    print(mark)
    print('')
    print('')
    #if statement to break the while loop.
    if trades != []:
        #starting a for loop to append the elements we need from each trade to our lists
        for e in trades:
            #adding to the counter.
            count +=1
            # if statement so we just scrap elements before today.
            if e.created_at < now :
                #printing each trade so we can see that everythings is working fine.
                print(e)
                #scrapping the elements we need
                btc_mxn_price.append(e.price)
                btc_mxn_volume.append(e.amount)
                btc_mxn_hour.append(e.created_at.hour)
                btc_mxn_date.append(e.created_at.date())
                btc_mxn_side.append(e.maker_side)
        '''
        if statement that starts a timer when the count reachs 100, we can just get
        300 requests per minute so our timer waits 21 seconds before making the next
        100 requests. It also adds a thousand to our mark so it moves forward in time.
        '''
        if count == 100:
            time.sleep(21)
            count = 0
            mark = mark + 1000
            
    #else statement to break the loop.
    else:
        break
        

13337280


Trade(tid=13337283, price=160350.59, amount=0.00060727, maker_side=buy, created_at=2020-01-26 23:50:05+00:00)
Trade(tid=13337284, price=160351.01, amount=0.00941568, maker_side=buy, created_at=2020-01-26 23:50:17+00:00)
Trade(tid=13337289, price=160352.12, amount=0.00005650, maker_side=buy, created_at=2020-01-26 23:50:47+00:00)
Trade(tid=13337290, price=161300.63, amount=0.02254600, maker_side=sell, created_at=2020-01-26 23:50:48+00:00)
Trade(tid=13337291, price=161300.63, amount=0.02760112, maker_side=sell, created_at=2020-01-26 23:50:48+00:00)
Trade(tid=13337292, price=161300.64, amount=0.05296758, maker_side=sell, created_at=2020-01-26 23:50:48+00:00)
Trade(tid=13337293, price=161300.64, amount=0.00030998, maker_side=sell, created_at=2020-01-26 23:51:42+00:00)
Trade(tid=13337299, price=161300.64, amount=0.01474272, maker_side=sell, created_at=2020-01-26 23:53:11+00:00)
Trade(tid=13337300, price=161300.64, amount=0.00012399, maker_side=sell, created_at=2020-01-26 23:53:12+

13338280


Trade(tid=13338295, price=162497.51, amount=0.02338497, maker_side=sell, created_at=2020-01-27 01:02:38+00:00)
Trade(tid=13338301, price=162497.05, amount=0.00309535, maker_side=sell, created_at=2020-01-27 01:03:57+00:00)
Trade(tid=13338302, price=162002.22, amount=0.00062132, maker_side=buy, created_at=2020-01-27 01:04:05+00:00)
Trade(tid=13338309, price=162496.71, amount=0.01720686, maker_side=sell, created_at=2020-01-27 01:04:50+00:00)
Trade(tid=13338310, price=162496.71, amount=0.01811248, maker_side=sell, created_at=2020-01-27 01:04:50+00:00)
Trade(tid=13338311, price=162496.72, amount=0.11237595, maker_side=sell, created_at=2020-01-27 01:04:50+00:00)
Trade(tid=13338312, price=162496.88, amount=0.00363155, maker_side=sell, created_at=2020-01-27 01:04:55+00:00)
Trade(tid=13338313, price=162496.89, amount=0.01975351, maker_side=sell, created_at=2020-01-27 01:04:55+00:00)
Trade(tid=13338314, price=162002.64, amount=0.00237024, maker_side=buy, created_at=2020-01-27 01:05:06

13339280


Trade(tid=13339281, price=162106.55, amount=0.00031046, maker_side=buy, created_at=2020-01-27 02:36:31+00:00)
Trade(tid=13339286, price=162789.99, amount=0.00123976, maker_side=sell, created_at=2020-01-27 02:37:14+00:00)
Trade(tid=13339287, price=162789.99, amount=0.00225812, maker_side=sell, created_at=2020-01-27 02:37:15+00:00)
Trade(tid=13339290, price=162789.99, amount=0.00614288, maker_side=sell, created_at=2020-01-27 02:37:50+00:00)
Trade(tid=13339291, price=162107.27, amount=0.04339522, maker_side=buy, created_at=2020-01-27 02:37:54+00:00)
Trade(tid=13339292, price=162107.27, amount=0.00944352, maker_side=buy, created_at=2020-01-27 02:37:54+00:00)
Trade(tid=13339294, price=162107.40, amount=0.00708943, maker_side=buy, created_at=2020-01-27 02:38:10+00:00)
Trade(tid=13339305, price=162499.99, amount=0.00003077, maker_side=sell, created_at=2020-01-27 02:40:32+00:00)
Trade(tid=13339308, price=162108.42, amount=0.00120000, maker_side=buy, created_at=2020-01-27 02:41:10+00

13340280


Trade(tid=13340281, price=162116.04, amount=0.01342567, maker_side=buy, created_at=2020-01-27 05:41:24+00:00)
Trade(tid=13340284, price=162369.94, amount=0.01039601, maker_side=sell, created_at=2020-01-27 05:41:46+00:00)
Trade(tid=13340286, price=162116.04, amount=0.00006209, maker_side=buy, created_at=2020-01-27 05:42:02+00:00)
Trade(tid=13340292, price=162116.04, amount=0.00006209, maker_side=buy, created_at=2020-01-27 05:43:12+00:00)
Trade(tid=13340293, price=162369.94, amount=0.00061588, maker_side=sell, created_at=2020-01-27 05:43:27+00:00)
Trade(tid=13340301, price=162116.04, amount=0.01155155, maker_side=buy, created_at=2020-01-27 05:44:15+00:00)
Trade(tid=13340309, price=162369.94, amount=0.02218081, maker_side=sell, created_at=2020-01-27 05:46:42+00:00)
Trade(tid=13340310, price=162369.94, amount=0.00010662, maker_side=sell, created_at=2020-01-27 05:46:42+00:00)
Trade(tid=13340311, price=162370.00, amount=0.00064361, maker_side=sell, created_at=2020-01-27 05:46:42+0

13341280


Trade(tid=13341297, price=162401.41, amount=0.02310200, maker_side=buy, created_at=2020-01-27 11:47:34+00:00)
Trade(tid=13341320, price=162554.71, amount=0.10000000, maker_side=buy, created_at=2020-01-27 11:49:24+00:00)
Trade(tid=13341331, price=162700.00, amount=0.25000000, maker_side=sell, created_at=2020-01-27 11:49:48+00:00)
Trade(tid=13341389, price=162699.99, amount=0.02328200, maker_side=sell, created_at=2020-01-27 11:51:15+00:00)
Trade(tid=13341395, price=162699.99, amount=0.12119000, maker_side=sell, created_at=2020-01-27 11:51:16+00:00)
Trade(tid=13341396, price=162699.99, amount=0.02328200, maker_side=sell, created_at=2020-01-27 11:51:17+00:00)
Trade(tid=13341402, price=162700.00, amount=0.02328200, maker_side=sell, created_at=2020-01-27 11:51:19+00:00)
Trade(tid=13341410, price=162700.00, amount=0.02328200, maker_side=sell, created_at=2020-01-27 11:51:21+00:00)
Trade(tid=13341411, price=162700.00, amount=0.14354825, maker_side=sell, created_at=2020-01-27 11:51:22

...

13397280


Trade(tid=13397286, price=177700.99, amount=0.34589257, maker_side=buy, created_at=2020-01-31 01:41:31+00:00)
Trade(tid=13397289, price=177700.99, amount=0.00031770, maker_side=buy, created_at=2020-01-31 01:42:00+00:00)
Trade(tid=13397290, price=177662.96, amount=0.02556733, maker_side=buy, created_at=2020-01-31 01:42:00+00:00)
Trade(tid=13397291, price=177662.96, amount=0.00002833, maker_side=buy, created_at=2020-01-31 01:42:19+00:00)
Trade(tid=13397293, price=177662.96, amount=0.17464000, maker_side=buy, created_at=2020-01-31 01:42:30+00:00)
Trade(tid=13397294, price=177662.96, amount=0.01133093, maker_side=buy, created_at=2020-01-31 01:42:35+00:00)
Trade(tid=13397298, price=177662.96, amount=0.00085009, maker_side=buy, created_at=2020-01-31 01:42:54+00:00)
Trade(tid=13397299, price=177790.22, amount=0.00065538, maker_side=sell, created_at=2020-01-31 01:42:54+00:00)
Trade(tid=13397303, price=177662.96, amount=0.00849820, maker_side=buy, created_at=2020-01-31 01:43:15+00:00

13398280


Trade(tid=13398282, price=176077.84, amount=0.00056793, maker_side=sell, created_at=2020-01-31 03:10:08+00:00)
Trade(tid=13398286, price=176077.84, amount=0.00056794, maker_side=sell, created_at=2020-01-31 03:10:11+00:00)
Trade(tid=13398289, price=176077.84, amount=0.00567931, maker_side=sell, created_at=2020-01-31 03:10:29+00:00)
Trade(tid=13398293, price=176077.84, amount=0.05679307, maker_side=sell, created_at=2020-01-31 03:10:44+00:00)
Trade(tid=13398296, price=176077.84, amount=0.00042595, maker_side=sell, created_at=2020-01-31 03:11:08+00:00)
Trade(tid=13398300, price=176031.75, amount=0.00118728, maker_side=buy, created_at=2020-01-31 03:11:44+00:00)
Trade(tid=13398305, price=176077.84, amount=0.01370442, maker_side=sell, created_at=2020-01-31 03:12:19+00:00)
Trade(tid=13398306, price=176177.00, amount=0.01000000, maker_side=sell, created_at=2020-01-31 03:12:19+00:00)
Trade(tid=13398307, price=176200.00, amount=0.01655728, maker_side=sell, created_at=2020-01-31 03:12:1

13399280


Trade(tid=13399281, price=175999.99, amount=0.00607451, maker_side=sell, created_at=2020-01-31 04:48:09+00:00)
Trade(tid=13399285, price=175920.57, amount=0.04864311, maker_side=sell, created_at=2020-01-31 04:48:29+00:00)
Trade(tid=13399286, price=175920.57, amount=0.05684384, maker_side=sell, created_at=2020-01-31 04:48:32+00:00)
Trade(tid=13399287, price=175824.46, amount=0.04832693, maker_side=buy, created_at=2020-01-31 04:48:55+00:00)
Trade(tid=13399295, price=175920.57, amount=0.00170532, maker_side=sell, created_at=2020-01-31 04:50:22+00:00)
Trade(tid=13399298, price=175824.46, amount=0.00368121, maker_side=buy, created_at=2020-01-31 04:50:38+00:00)
Trade(tid=13399304, price=175920.57, amount=0.00568438, maker_side=sell, created_at=2020-01-31 04:51:07+00:00)
Trade(tid=13399314, price=175824.46, amount=0.00057248, maker_side=buy, created_at=2020-01-31 04:52:08+00:00)
Trade(tid=13399327, price=175824.46, amount=0.00896761, maker_side=buy, created_at=2020-01-31 04:52:33+0

13400280


Trade(tid=13400281, price=174985.00, amount=0.00285739, maker_side=buy, created_at=2020-01-31 07:23:34+00:00)
Trade(tid=13400282, price=174970.78, amount=0.25000000, maker_side=buy, created_at=2020-01-31 07:23:34+00:00)
Trade(tid=13400289, price=174900.50, amount=0.00057550, maker_side=buy, created_at=2020-01-31 07:26:43+00:00)
Trade(tid=13400290, price=174900.50, amount=0.01463355, maker_side=buy, created_at=2020-01-31 07:27:04+00:00)
Trade(tid=13400293, price=174900.50, amount=0.00194356, maker_side=buy, created_at=2020-01-31 07:27:48+00:00)
Trade(tid=13400294, price=174837.80, amount=0.13647644, maker_side=buy, created_at=2020-01-31 07:27:48+00:00)
Trade(tid=13400295, price=174885.65, amount=0.00058238, maker_side=sell, created_at=2020-01-31 07:27:57+00:00)
Trade(tid=13400301, price=174885.64, amount=0.00285907, maker_side=sell, created_at=2020-01-31 07:31:38+00:00)
Trade(tid=13400302, price=174800.00, amount=0.00057583, maker_side=buy, created_at=2020-01-31 07:33:55+00:0

13401280


Trade(tid=13401286, price=174505.10, amount=0.00017304, maker_side=buy, created_at=2020-01-31 12:35:04+00:00)
Trade(tid=13401364, price=174505.10, amount=0.01561458, maker_side=buy, created_at=2020-01-31 12:41:27+00:00)
Trade(tid=13401385, price=174501.99, amount=0.10454000, maker_side=buy, created_at=2020-01-31 12:41:34+00:00)
Trade(tid=13401386, price=174500.00, amount=0.23869086, maker_side=buy, created_at=2020-01-31 12:41:34+00:00)
Trade(tid=13401405, price=174500.00, amount=0.04206658, maker_side=buy, created_at=2020-01-31 12:41:42+00:00)
Trade(tid=13401407, price=174501.99, amount=0.00085959, maker_side=sell, created_at=2020-01-31 12:41:43+00:00)
Trade(tid=13401443, price=174500.00, amount=0.00006027, maker_side=buy, created_at=2020-01-31 12:41:55+00:00)
Trade(tid=13401445, price=174415.11, amount=0.25000000, maker_side=buy, created_at=2020-01-31 12:41:55+00:00)
Trade(tid=13401446, price=174300.00, amount=0.10279726, maker_side=buy, created_at=2020-01-31 12:41:55+00:00

In [4]:
# creating a new data frame.
df = pd.DataFrame(columns=['price', 'volume', 'hour', 'date', 'side'])
# using the lists we got from the while loop to fill the dataframe.
df.price = btc_mxn_price
df.volume = btc_mxn_volume
df.hour = btc_mxn_hour
df.date = btc_mxn_date
df.side = btc_mxn_side
df.date = df.date.astype('datetime64')
#creating new columns with the month and day of each row.
df['month'] = [i.month for i in df.date]
df['day'] = [i.day for i in df.date]

In [5]:
#Creating a function that needs a data frame and a number(month) for input.
def extraction(df, m):
    #variable that get's just the row that matches the same month
    f_df = df.loc[df['month'] == m]
    #variable that stores the year
    y = datetime.datetime.today().year
    #variable that creates a list with the numbers of days of the month
    days = list(f_df['day'].unique())
    #varible that creates a empty dictionary
    results = {}
    #list with the names of the keys in the dictionary that the function returns.
    nombres = ['date', '24h_avg', '12h_avg', 'min', 'max', 'volume', 'm_side']
    
    #for loop that creates new keys in the empty dictionary with a empty list as values.
    for i in nombres:
        results[i] = []
    
    #for that loops on each day in the list of the days of the month.
    for e in days:
        #variable that get's just the rows that match the day in the for loop.
        ndf = f_df.loc[f_df.day == e]
        #appending the results in each dictionary key.
        results['date'].append(datetime.date(y, m, e))
        results['24h_avg'].append(ndf.price.mean())
        results['12h_avg'].append(ndf.price.loc[ndf.hour<12].mean())
        results['max'].append(ndf.price.max())
        results['min'].append(ndf.price.min())
        results['volume'].append(ndf.volume.sum())
        weight = ndf.side.value_counts().sum()
        buy = ndf.side.value_counts()['buy']/weight
        sell = ndf.side.value_counts()['sell']/weight
        if buy>sell:
            results['m_side'].append('buy')
        else:
            results['m_side'].append('sell')
            
    #returns the dictionary with the appended elements
    return results

In [6]:
#creating a variable that stores the month we are in.
month = datetime.datetime.today().month
#creating a new data frame using the dictionary from the above function and the month
df_new = pd.DataFrame.from_dict(extraction(df, month))

In [7]:
#Reading our csv file where we store the past data
history = pd.read_csv('../csv_files/historic_data.csv')

In [8]:
btc_df = pd.concat([history, df_new], ignore_index=True)

In [9]:
btc_df.drop_duplicates(inplace=True)

In [10]:
btc_df.drop(btc_df.tail(1).index,inplace=True)

In [11]:
btc_df.to_csv('../csv_files/historic_data.csv', index=False)

In [12]:
btc_df.date = btc_df.date.astype('datetime64')
btc_df['min'] = btc_df['min'].astype('float')
btc_df['max'] = btc_df['max'].astype('float')
btc_df.volume = btc_df.volume.astype('float')
btc_df['m_side'] = btc_df['m_side'].astype('category')

In [13]:
df_dum = pd.get_dummies(btc_df)
df_dum.rename(columns={'m_side_buy':'buy', 'm_side_sell':'sell'}, inplace=True)

In [14]:
change = []
for i in range(1,len(df_dum)):
    if df_dum.sell.iloc[i] != df_dum.sell.iloc[i-1]:
        change.append(df_dum.date.iloc[i])

In [15]:
#Selecting the columns we need for fbprophet
btc_24h_avg = btc_df[['date', '24h_avg']].copy()
#Changing the columns names for fbprophet to accept it.
btc_24h_avg.rename(columns={'date': 'ds', '24h_avg': 'y'}, inplace=True)

In [16]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [17]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [18]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    df.dropna(inplace=True)
    
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [19]:
train_df = btc_24h_avg.copy().set_index('ds')

In [20]:
train_df['y'], lambda_prophet = boxcox(train_df['y'])
train_df.reset_index(inplace=True)

In [21]:
m2 = Prophet(changepoints=change, seasonality_mode='multiplicative')
m2.fit(train_df)
future2 = m2.make_future_dataframe(periods=15)
forecast2 = m2.predict(future2)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [22]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column],
                                       lambda_prophet)

In [23]:
cmp_df2 = make_comparison_dataframe(btc_24h_avg, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, 15).items():
     print(err_name, err_value)

MAPE 6.525778655466187
MAE 10819.996442615091


In [24]:


def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Zona de compra', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="green"))
    upper_bound = create_go('Máximo esperado', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="red"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Venta sugerida', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Precio promedio del día', 'y', num_values,
                       marker=dict(color="red"))
    
    
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='MXN'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename='btc_mxn.html', auto_open=False)

show_forecast(cmp_df2, 15, 2123, 'BTC')

